<a href="https://colab.research.google.com/github/its-serah/AI-Brand-Creator/blob/logo-generation-first-experiments/models_trial_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1- Logo Diffusion (LogoWizard)

Type & Features: Stable Diffusion 2.1 fine-tuned on logos – text-to image generator for logo icons . Prefers flat, centered designs.

Multilingual support: Prompt in EN (OpenCLIP may handle other languages); doesn’t generate readable text in images.

[logo diffusion](https://huggingface.co/logo-wizard/logo-diffusion-checkpoint#:~:text=This%20is%20checkpoint%20based%20on,example%20images%20in%20the%20following)

2- T2I-Adapter (Color)

Type & Features:  Auxiliary model for Stable Diffusion conditions generation on a provided color palette image . Ensures output image follows specific colors.

Multilingual support: N/A (used with SD text prompts, primarily English).

[T2I-Adapter](https://huggingface.co/TencentARC/t2iadapter_color_sd14v1#:~:text=T2I%20Adapter%20is%20a%20network,specific%20base%20stable%20diffusion%20checkpoint)


Tips for this pair

Resolutions:

logo-diffusion (SD-2.1) → 768×768 is native.
Hugging Face

t2iadapter_color_sd14v1 + SD-1.4 → 512×512 is native; you can upscale later.

Color strength (Model 2): tweak adapter_conditioning_scale (try 0.6, 1.0, 1.2).
Hugging Face

Prompting: keep it logo-like (“flat vector, simple, solid fills”), avoid photoreal cues.

Fairness: keep same SEED, STEPS, and guidance across models.

Step0: GPU check

In [ ]:
import torch, sys
print("Python:", sys.version)
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())

Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Torch: 2.8.0+cu126 | CUDA available: True


Step1: Installs

In [ ]:

%pip install -q --upgrade "transformers==4.45.2" "tokenizers==0.20.3" "diffusers==0.29.2" accelerate safetensors
%pip install -q pillow numpy pandas scikit-image tqdm open-clip-torch peft==0.13.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00


Step2: Imports and basic setup

In [ ]:
import os, math, time, random
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from tqdm import tqdm

import torch
from diffusers import (
    AutoPipelineForText2Image,
    StableDiffusionPipeline,
    StableDiffusionAdapterPipeline,
    T2IAdapter,
)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())

RUN_TAG = time.strftime("%Y%m%d-%H%M%S")
BASE_DIR = Path(f"/content/outputs/{RUN_TAG}")
BASE_DIR.mkdir(parents=True, exist_ok=True)
print("Outputs ->", BASE_DIR)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Torch: 2.8.0+cu126 | CUDA: True
Outputs -> /content/outputs/20250908-160041


/usr/local/lib/python3.12/dist-packages/diffusers/models/vq_model.py:20: FutureWarning: `VQEncoderOutput` is deprecated and will be removed in version 0.31. Importing `VQEncoderOutput` from `diffusers.models.vq_model` is deprecated and this will be removed in a future version. Please use `from diffusers.models.autoencoders.vq_model import VQEncoderOutput`, instead.
  deprecate("VQEncoderOutput", "0.31", deprecation_message)
/usr/local/lib/python3.12/dist-packages/diffusers/models/vq_model.py:25: FutureWarning: `VQModel` is deprecated and will be removed in version 0.31. Importing `VQModel` from `diffusers.models.vq_model` is deprecated and this will be removed in a future version. Please use `from diffusers.models.autoencoders.vq_model import VQModel`, instead.
  deprecate("VQModel", "0.31", deprecation_message)


Step3: Model 1 — Logo Diffusion (LogoWizard)

Loader

In [ ]:
from diffusers import StableDiffusionPipeline, AutoPipelineForText2Image

def load_logo_pipeline(model_id="logo-wizard/logo-diffusion-checkpoint"):
    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            use_safetensors=False,   # important for .bin
            safety_checker=None,
            local_files_only=False,
            low_cpu_mem_usage=False  # safer with .bin on Colab
        )
    except Exception:
        pipe = AutoPipelineForText2Image.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            use_safetensors=False,
            safety_checker=None,
            local_files_only=False,
            low_cpu_mem_usage=False
        )
    pipe.to(DEVICE)
    pipe.enable_attention_slicing(); pipe.enable_vae_slicing()
    return pipe

pipe1 = load_logo_pipeline()
print("Model 1 ready.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

text_encoder/pytorch_model.bin:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--logo-wizard--logo-diffusion-checkpoint/snapshots/4795d8af0496e9ab40006281fb5b237c23f6c412/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--logo-wizard--logo-diffusion-checkpoint/snapshots/4795d8af0496e9ab40006281fb5b237c23f6c412/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--logo-wizard--logo-diffusion-checkpoint/snapshots/4795d8af0496e9ab40006281fb5b237c23f6c412/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--logo-wizard--logo-diffusion-checkpoint/snapshots/4795d8af0496e9ab40006281fb5b237c23f6c412/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Model 1 ready.


Quick Smoke test

In [ ]:
prompt = "Cedar & Stone logo, minimal geometric emblem for premium Lebanese coffee roastery, flat vector, simple, clean, solid fills"
neg = "photorealistic, 3d render, busy background, text, watermark, clutter, gradients, bevel, shadow, noisy"

img = pipe1(
    prompt=prompt, negative_prompt=neg,
    height=512, width=512, num_inference_steps=15, guidance_scale=7.0
).images[0]

test_dir = BASE_DIR / "quick_test"; test_dir.mkdir(parents=True, exist_ok=True)
out1 = str(test_dir / "logo_diffusion_test.png")
img.save(out1); print("Saved:", out1)


  0%|          | 0/15 [00:00<?, ?it/s]

Saved: /content/outputs/20250908-160041/quick_test/logo_diffusion_test.png


✅ Strengths:
• It produced a clean, vector-like outline (good for logos).
• Strong silhouette contrast (the dark trees/shapes against the beige).
• Simple shape — scalable and recognizable.

⚠️ Weaknesses / points to improve:
• The design feels a bit generic (like a clipart badge, not strongly brand-unique).
• Some unwanted shading/textures inside the badge (logos should usually be flat/solid fills).
• The “top house/barn” shape might not match your “coffee roastery” or “brand story” unless that was intended.

💡 Next steps if you want better:
• Strengthen the prompt with style keywords: “flat minimal vector, no gradients, no shading, clean lines, solid fills, geometric coffee emblem”.
• Add industry hint: “coffee beans, roastery, Lebanese heritage”.
• If you want color adherence, test with Model 2 (T2I-Adapter Color) and provide your palette — it will force stronger brand colors.

👉 Verdict: it’s usable as a first pass, but you’d likely want sharper, flatter, and more on-theme logos.

In [ ]:
#prompt 2
prompt = (
    "Cedar & Stone logo, minimal flat vector emblem for premium Lebanese coffee roastery, "
    "geometric design, coffee beans and cedar tree, clean lines, simple, scalable, solid fills, "
    "modern branding style"
)

negative_prompt = (
    "photorealistic, 3d render, shading, gradients, bevel, texture, busy background, "
    "clutter, text, watermark, noisy, clipart, stock icon"
)

img = pipe1(
    prompt=prompt,
    negative_prompt=negative_prompt,
    height=512, width=512,   # keep small to test first
    num_inference_steps=25,
    guidance_scale=7.5
).images[0]

test_dir = BASE_DIR / "quick_test"
test_dir.mkdir(parents=True, exist_ok=True)
out1b = str(test_dir / "logo_diffusion_refined.png")
img.save(out1b); print("Saved:", out1b)


  0%|          | 0/25 [00:00<?, ?it/s]

Saved: /content/outputs/20250908-160041/quick_test/logo_diffusion_refined.png


In [ ]:
#trial 3
prompt = (
    "Cedar & Stone emblem-only logo symbol, no text, flat minimal vector, "
    "coffee beans merging with cedar tree silhouette, simple and modern, "
    "solid fills, scalable design, color palette of dark brown, beige, forest green"
)

negative_prompt = (
    "text, typography, font, words, writing, captions, letters, watermark, clutter, "
    "photorealistic, 3d render, gradients, shading, bevel, texture, noisy"
)

img = pipe1(
    prompt=prompt,
    negative_prompt=negative_prompt,
    height=512, width=512,   # test size first
    num_inference_steps=25,
    guidance_scale=7.5
).images[0]

test_dir = BASE_DIR / "quick_test"
test_dir.mkdir(parents=True, exist_ok=True)
out1c = str(test_dir / "logo_diffusion_textfree.png")
img.save(out1c); print("Saved:", out1c)


  0%|          | 0/25 [00:00<?, ?it/s]

Saved: /content/outputs/20250908-160041/quick_test/logo_diffusion_textfree.png


Now this is a much better step forward ✅

🔹 Strengths

No text at all → the “emblem-only” instruction worked.

Flat, vector-like fills → much cleaner than the previous versions.

Good use of symbolic elements → leaves, tree-like base, and circular framing → feels logo-ish.

Colors → strong brown + beige tones, some green hints → closer to a brand palette.

🔹 Weaknesses

The lower half tree still has a “speckled / textured” look, not clean solid fills.

Some leaf details are a bit busy / noisy → can be simplified more for scalability.

Circle outline is slightly broken / uneven.

🔹 Verdict

This output is finally usable as a logo draft — you can easily take this into Illustrator/Figma, trace/clean it, and get a professional-quality emblem.

Next Steps:

Prompt tweak to reduce texture:
Add → negative_prompt = "... texture, dots, stippling, detailed noise"
And emphasize → "solid flat fills, no small details".

In [ ]:
#next prompt trial
prompt = (
    "Cedar & Stone emblem-only logo symbol, minimal flat vector style, "
    "coffee beans merging with a cedar tree silhouette, circular emblem, "
    "geometric, simple, scalable, solid flat fills, "
    "modern professional branding, color palette of dark brown, beige, and forest green"
)

negative_prompt = (
    "text, typography, font, words, writing, letters, watermark, "
    "texture, dots, stippling, grain, noise, shading, gradients, bevel, emboss, "
    "photorealistic, 3d render, busy, clutter"
)

img = pipe1(
    prompt=prompt,
    negative_prompt=negative_prompt,
    height=512, width=512,   # start small, upscale later
    num_inference_steps=30,  # more steps = cleaner detail
    guidance_scale=8.0       # stronger adherence to prompt
).images[0]

test_dir = BASE_DIR / "quick_test"
test_dir.mkdir(parents=True, exist_ok=True)
out1d = str(test_dir / "logo_diffusion_flat.png")
img.save(out1d); print("Saved:", out1d)


  0%|          | 0/30 [00:00<?, ?it/s]

Saved: /content/outputs/20250908-160041/quick_test/logo_diffusion_flat.png


Better but shape is still not wow hahah

In [ ]:
#Switch to a sampler that’s great for crisp, flat shapes
from diffusers import EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler

# Try Euler A first (often best for logo-like crispness). If you prefer, swap to DPM-Solver after.
pipe1.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe1.scheduler.config)
# Alternative:
# pipe1.scheduler = DPMSolverMultistepScheduler.from_config(pipe1.scheduler.config)


In [ ]:
#Use an ultra-flat, emblem-only prompt
POS = (
    "Cedar & Stone emblem-only logo symbol, flat 2D vector icon, "
    "cedar tree + coffee bean fusion, circular badge, bold silhouette, "
    "simple geometry, thick clean outline, solid flat fills, no lighting, no shadows, "
    "modern professional branding, color palette dark brown, beige, forest green"
)

NEG = (
    "text, typography, letters, words, writing, watermark, caption, "
    "photorealistic, 3d, perspective, lighting, shadow, highlight, "
    "gradients, shading, texture, dots, stippling, grain, noise, bevel, emboss, glow, clutter, background scene"
)


In [ ]:
#Generate a small grid of variations (different seeds) and pick the cleanest
import random, os
random_seeds = [11, 22, 33, 44, 55, 66]  # change if you want different looks
grid_dir = BASE_DIR / "model1_ultraflat"; grid_dir.mkdir(parents=True, exist_ok=True)

def run_one(seed, size=640, steps=20, guidance=6.5):
    generator = torch.Generator(device=pipe1.device).manual_seed(seed)
    img = pipe1(
        prompt=POS,
        negative_prompt=NEG,
        height=size, width=size,                 # go 640 first; bump to 768 if VRAM is fine
        num_inference_steps=steps,
        guidance_scale=guidance,
        generator=generator
    ).images[0]
    out = grid_dir / f"m1_ultraflat_s{seed}_{size}.png"
    img.save(out); return out

outs = [run_one(s) for s in random_seeds]
outs


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

[PosixPath('/content/outputs/20250908-160041/model1_ultraflat/m1_ultraflat_s11_640.png'),
 PosixPath('/content/outputs/20250908-160041/model1_ultraflat/m1_ultraflat_s22_640.png'),
 PosixPath('/content/outputs/20250908-160041/model1_ultraflat/m1_ultraflat_s33_640.png'),
 PosixPath('/content/outputs/20250908-160041/model1_ultraflat/m1_ultraflat_s44_640.png'),
 PosixPath('/content/outputs/20250908-160041/model1_ultraflat/m1_ultraflat_s55_640.png'),
 PosixPath('/content/outputs/20250908-160041/model1_ultraflat/m1_ultraflat_s66_640.png')]

Step 4: Model 2 - T2I-Adapter (Color) + SD-1.4 base

step 4.1 - Loader

In [ ]:
# Base (SD 1.4) + Color Adapter
from diffusers import StableDiffusionAdapterPipeline, T2IAdapter
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ADAPTER_ID = "TencentARC/t2iadapter_color_sd14v1"
BASE_SD = "CompVis/stable-diffusion-v1-4"  # trained target for this adapter

adapter = T2IAdapter.from_pretrained(
    ADAPTER_ID,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(DEVICE)

pipe2 = StableDiffusionAdapterPipeline.from_pretrained(
    BASE_SD,
    adapter=adapter,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    safety_checker=None
).to(DEVICE)

# memory helpers (we skipped xformers)
pipe2.enable_attention_slicing(); pipe2.enable_vae_slicing()
print("Model 2 ready.")


config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

An error occurred while trying to fetch TencentARC/t2iadapter_color_sd14v1: TencentARC/t2iadapter_color_sd14v1 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


diffusion_pytorch_model.bin:   0%|          | 0.00/74.8M [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.t2i_adapter.pipeline_stable_diffusion_adapter.StableDiffusionAdapterPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Model 2 ready.


Step 4.2 - Palette helper

In [ ]:
from PIL import Image, ImageDraw

def hex_to_rgb(h): h=h.strip().lstrip("#"); return tuple(int(h[i:i+2],16) for i in (0,2,4))
def make_palette_image(hex_list, grid=4, cell=64, pad=0):
    W = grid*cell + (grid+1)*pad; H = W
    img = Image.new("RGB", (W,H), (255,255,255)); draw = ImageDraw.Draw(img)
    cols = [hex_to_rgb(x) for x in (hex_list or ["#000000", "#FFFFFF"])]
    k=0
    for r in range(grid):
        for c in range(grid):
            x0=c*cell+(c+1)*pad; y0=r*cell+(r+1)*pad
            draw.rectangle([x0,y0,x0+cell,y0+cell], fill=cols[k%len(cols)])
            k+=1
    return img

# your Lebanese coffee palette (tweak as you like)
PALETTE = ["#0A2540", "#2E7D6F", "#6B3E2E", "#E8F5F2", "#E5D6B0"]  # deep navy, cedar green, coffee brown, mint, sand
palette_img = make_palette_image(PALETTE, grid=4, cell=64, pad=0)


Step 4.3 - Ultraflat prompts (text-free, emblem-only)

In [ ]:
POS2 = (
    "Cedar & Stone emblem-only logo symbol, flat 2D vector icon, "
    "cedar tree + coffee bean fusion, circular badge, bold silhouette, "
    "simple geometry, thick clean outline, solid flat fills, no lighting, no shadows, "
    "modern professional branding"
)
NEG2 = (
    "text, typography, letters, words, writing, watermark, caption, "
    "photorealistic, 3d, perspective, lighting, shadow, highlight, "
    "gradients, shading, texture, dots, stippling, grain, noise, bevel, emboss, glow, clutter, background scene"
)


Step 4.4 - Generate a small seed sweep (pick your favorite)

In [ ]:
import os
seeds = [101, 202, 303, 404, 505, 606]
m2_dir = BASE_DIR / "model2_color"; m2_dir.mkdir(parents=True, exist_ok=True)

def run_adapter(seed, size=512, steps=22, guidance=6.5, cond=1.0):
    g = torch.Generator(device=pipe2.device).manual_seed(seed)
    img = pipe2(
        prompt=POS2,
        negative_prompt=NEG2,
        image=palette_img,                     # <-- color conditioning
        height=size, width=size,               # SD1.x native 512; upscale later if needed
        num_inference_steps=steps,
        guidance_scale=guidance,
        adapter_conditioning_scale=cond,       # try 0.6–1.4; higher = stronger palette lock
        generator=g
    ).images[0]
    outp = m2_dir / f"m2_color_s{seed}_{size}_c{cond}.png"
    img.save(outp); return outp

outs2 = [run_adapter(s, size=512, steps=22, guidance=6.5, cond=1.0) for s in seeds]
outs2


  0%|          | 0/22 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/diffusers/pipelines/t2i_adapter/pipeline_stable_diffusion_adapter.py:500: FutureWarning: The decode_latents method is deprecated and will be removed in 1.0.0. Please use VaeImageProcessor.postprocess(...) instead
  deprecate("decode_latents", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

[PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s101_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s202_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s303_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s404_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s505_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s606_512_c1.0.png')]

In [ ]:
#prompt tuning for structure
#Add stronger shape + concept language in POS2
POS2 = (
    "Cedar & Stone emblem-only logo symbol, flat 2D vector icon, "
    "fusion of cedar tree silhouette and coffee bean, "
    "bold circular badge, geometric balance, strong symmetry, "
    "minimalist professional branding, solid flat fills, no shading"
)
NEG2 = (
    "text, typography, letters, words, writing, watermark, captions, ornate, filigree, "
    "photorealistic, 3d, gradients, shading, texture, dots, stippling, grain, "
    "abstract app icon, rounded square background"
)


In [ ]:
import os
seeds = [101, 202, 303, 404, 505, 606]
m2_dir = BASE_DIR / "model2_color"; m2_dir.mkdir(parents=True, exist_ok=True)

def run_adapter(seed, size=512, steps=22, guidance=6.5, cond=1.0):
    g = torch.Generator(device=pipe2.device).manual_seed(seed)
    img = pipe2(
        prompt=POS2,
        negative_prompt=NEG2,
        image=palette_img,                     # <-- color conditioning
        height=size, width=size,               # SD1.x native 512; upscale later if needed
        num_inference_steps=steps,
        guidance_scale=guidance,
        adapter_conditioning_scale=cond,       # try 0.6–1.4; higher = stronger palette lock
        generator=g
    ).images[0]
    outp = m2_dir / f"m2_color_s{seed}_{size}_c{cond}.png"
    img.save(outp); return outp

outs2 = [run_adapter(s, size=512, steps=22, guidance=6.5, cond=1.0) for s in seeds]
outs2


  0%|          | 0/22 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/diffusers/pipelines/t2i_adapter/pipeline_stable_diffusion_adapter.py:500: FutureWarning: The decode_latents method is deprecated and will be removed in 1.0.0. Please use VaeImageProcessor.postprocess(...) instead
  deprecate("decode_latents", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

[PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s101_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s202_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s303_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s404_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s505_512_c1.0.png'),
 PosixPath('/content/outputs/20250908-160041/model2_color/m2_color_s606_512_c1.0.png')]

In [ ]:
##REFINED MODEL 2
#stronger prompts # Strong structure: cedar + coffee fusion, emblem-only, flat
POS2_IMPROVED = (
    "Minimalist emblem-only logo symbol, fusion of a cedar tree silhouette and a coffee bean, "
    "flat 2D vector icon, bold circular badge, strong symmetry, simple geometric forms, "
    "thick clean outline, solid flat fills, modern professional branding"
)

# Hard bans: no text, no monograms, no rounded-square app icons, no background textures
NEG2_IMPROVED = (
    "text, typography, letters, words, writing, watermark, monogram, initials, "
    "ornate, filigree, thin lines, tiny details, abstract random symbol, "
    "app icon, rounded square background, badge with outer text ring, "
    "background texture, wood grain, fabric, gradient, shading, lighting, shadow, bevel, emboss, glow, noise, dots, stippling"
)


In [ ]:
#Parameter sweeps (conditioning & guidance)
# Small, focused sweep
seeds_m2   = [121, 232, 343, 454, 565, 676]   # change if you want new shapes
conds      = [0.8, 1.0, 1.2]                  # palette adherence strength
guidances  = [6.0, 7.0]                       # prompt adherence; 6.0=looser, 7.0=tighter
steps      = 24                               # sweet spot for SD1.x + adapter
size       = 512                              # render best later at 768

m2_imp_dir = BASE_DIR / "model2_color_improved"
m2_imp_dir.mkdir(parents=True, exist_ok=True)

def run_adapter_improved(seed, cond, guidance, size=size, steps=steps):
    g = torch.Generator(device=pipe2.device).manual_seed(seed)
    img = pipe2(
        prompt=POS2_IMPROVED,
        negative_prompt=NEG2_IMPROVED,
        image=palette_img,                 # <-- palette conditioning image you built earlier
        height=size, width=size,
        num_inference_steps=steps,
        guidance_scale=guidance,
        adapter_conditioning_scale=cond,
        generator=g
    ).images[0]
    outp = m2_imp_dir / f"m2_imp_s{seed}_c{cond}_g{guidance}_{size}.png"
    img.save(outp);
    return str(outp)

# Run the sweep
outs_m2 = []
for s in seeds_m2:
    for c in conds:
        for g in guidances:
            outs_m2.append(run_adapter_improved(s, c, g))
outs_m2[:6], len(outs_m2)


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

...

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

(['/content/outputs/20250908-160041/model2_color_improved/m2_imp_s121_c0.8_g6.0_512.png',
  '/content/outputs/20250908-160041/model2_color_improved/m2_imp_s121_c0.8_g7.0_512.png',
  '/content/outputs/20250908-160041/model2_color_improved/m2_imp_s121_c1.0_g6.0_512.png',
  '/content/outputs/20250908-160041/model2_color_improved/m2_imp_s121_c1.0_g7.0_512.png',
  '/content/outputs/20250908-160041/model2_color_improved/m2_imp_s121_c1.2_g6.0_512.png',
  '/content/outputs/20250908-160041/model2_color_improved/m2_imp_s121_c1.2_g7.0_512.png'],
 36)

In [ ]:
#Pick a favorite and upscale to 768
# Example: choose the seed/cond/guidance combo you like from the sweep
best_seed = 232
best_cond = 1.2     # stronger palette lock if colors drift
best_guid = 7.0     # tighter on structure
best_768  = run_adapter_improved(best_seed, best_cond, best_guid, size=768, steps=26)
print("Upscaled best:", best_768)

  0%|          | 0/26 [00:00<?, ?it/s]

Upscaled best: /content/outputs/20250908-160041/model2_color_improved/m2_imp_s232_c1.2_g7.0_768.png
